# Test and results.

In [1]:
from markovify.HMM import HMM
from markovify.ViterbiDecoder import ViterbiDecoder

from nltk.corpus import brown
from math import ceil, floor

import pandas as pd
import numpy as np

In [2]:
# Just a progress bar to know how a loop is progressing
# Code not related to the project.
# All credit to https://github.com/alexanderkuk/log-progress


def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

# Accuracy for news text.

In [3]:
corpus_tagged = brown.tagged_sents(categories='news', tagset='universal')
corpus_raw = brown.sents(categories='news')
size = len(corpus_tagged)

corpus_train = corpus_tagged[:floor(size*9/10)]
corpus_test_tagged = corpus_tagged[floor(size*9/10):]
corpus_test = corpus_raw[floor(size*9/10):]

print(len(corpus_train))
print(len(corpus_test))

4160
463


In [4]:
model = HMM(smoothing='laplace', alpha=1)
model.train(corpus_train)
#print(model._a)
#print(model.b)

decoder = ViterbiDecoder(model)

In [5]:
flat_list = [item for sublist in corpus_test_tagged for item in sublist]

results = pd.DataFrame.from_records(flat_list)
results.columns = ["Word", "Tag"]
results.head(10)

,Word,Tag
0,But,CONJ
1,in,ADP
2,all,PRT
3,its,DET
4,175,NUM
5,years,NOUN
6,",",.
7,not,ADV
8,a,DET
9,single,ADJ


In [6]:
tagged = []
for sent in log_progress(corpus_test, every=1):
    tagged.append(decoder.viterbi(sent))

In [7]:
flat_list = [item for sublist in tagged for item in sublist]

results['Predicted'] = flat_list
results.head(10)

,Word,Tag,Predicted
0,But,CONJ,CONJ
1,in,ADP,ADP
2,all,PRT,PRT
3,its,DET,DET
4,175,NUM,ADJ
5,years,NOUN,NOUN
6,",",.,.
7,not,ADV,ADV
8,a,DET,DET
9,single,ADJ,ADJ


In [8]:
total_words = results.shape[0]
correct = 0
for index, row in results.iterrows():
    if row[1] == row[2]:
        correct += 1
brown_news_accuracy = correct / total_words
print(brown_news_accuracy)

0.9120901026612179


# Accuracy for news text training the model using an adventures book.

In [9]:
corpus_tagged = brown.tagged_sents(categories='adventure', tagset='universal')
corpus_test_tagged = brown.tagged_sents(categories='news', tagset='universal')
corpus_test_raw = brown.sents(categories='news')

size = len(corpus_tagged)

corpus_train = corpus_tagged[:floor(size*9/10)]
corpus_test_tagged = corpus_test_tagged[:floor(size*1/10)]
corpus_test = corpus_test_raw[:floor(size*1/10)]

print(len(corpus_train))
print(len(corpus_test))

4173
463


In [10]:
model = HMM(smoothing='laplace', alpha=1)
model.train(corpus_train)

decoder = ViterbiDecoder(model)

tagged = []
for sent in log_progress(corpus_test, every=1):
    tagged.append(decoder.viterbi(sent))

In [11]:
flat_list = [item for sublist in corpus_test_tagged for item in sublist]

results = pd.DataFrame.from_records(flat_list)
results.columns = ["Word", "Tag"]

flat_list = [item for sublist in tagged for item in sublist]

results['Predicted'] = flat_list
results.head(10)

,Word,Tag,Predicted
0,The,DET,DET
1,Fulton,NOUN,ADJ
2,County,NOUN,NOUN
3,Grand,ADJ,ADP
4,Jury,NOUN,NOUN
5,said,VERB,VERB
6,Friday,NOUN,ADP
7,an,DET,DET
8,investigation,NOUN,NOUN
9,of,ADP,ADP


In [12]:
total_words = results.shape[0]
correct = 0
for index, row in results.iterrows():
    if row[1] == row[2]:
        correct += 1
brown_news_adventures_accuracy = correct / total_words
print(brown_news_adventures_accuracy)

0.8024051803885291


## Same experiment using $\alpha = 0.7$.

In [13]:
model = HMM(smoothing='laplace', alpha=0.7)
model.train(corpus_train)

decoder = ViterbiDecoder(model)

tagged = []
for sent in log_progress(corpus_test, every=1):
    tagged.append(decoder.viterbi(sent))

flat_list = [item for sublist in corpus_test_tagged for item in sublist]

results = pd.DataFrame.from_records(flat_list)
results.columns = ["Word", "Tag"]

flat_list = [item for sublist in tagged for item in sublist]

results['Predicted'] = flat_list
results.head(10)

total_words = results.shape[0]
correct = 0
for index, row in results.iterrows():
    if row[1] == row[2]:
        correct += 1
brown_news_adventures_accuracy_07 = correct / total_words
print(brown_news_adventures_accuracy_07)

0.7999074930619796
